# Migrate

In [1]:
import uuid
import pandas as pd
from db import MongoDB, PostgresDB

In [2]:
mongo = MongoDB().connect()
postgres = PostgresDB()

In [3]:
postgres.init_db()

Applied schema: 001_schema.sql


In [4]:
books = pd.DataFrame(list(mongo.get_collection('metadata_clean').find({})))
books.head()

,_id,title,author,description,image_url,reviews_count,rating,media_type,genres,year
0,73f65a70-8825-5401-bcce-2313da0cc6f8,Crime and Punishment,Fyodor Dostoevsky,Fyodor Dostoevsky's Crime and Punishment is on...,https://images-na.ssl-images-amazon.com/images...,521,4.25,"[Paper, Audiobook, Ebook]","[General Fiction, Fiction and Literature, Myst...",1866
1,e940f1b3-d333-50b7-a2b6-b95603134438,Love in the Time of Cholera,Gabriel García Márquez,"In their youth, Florentino Ariza and Fermina D...",https://images-na.ssl-images-amazon.com/images...,423,3.95,"[Paper, Audiobook, Ebook]","[General Fiction, Fiction and Literature, Roma...",1985
2,74ec4ed6-0c42-5425-86bf-51af8793a280,Mansfield Park,Jane Austen,Mansfield Park is the study of three families-...,https://images-na.ssl-images-amazon.com/images...,373,3.84,"[Paper, Audiobook, Ebook]","[Fiction and Literature, Romance, General Fict...",1814
3,8368087f-946e-5eb0-b630-a614124d550f,Siddhartha,Hermann Hesse,Hermann Hesse wrote Siddhartha after he travel...,https://images-na.ssl-images-amazon.com/images...,398,3.95,"[Paper, Audiobook, Ebook]","[Fiction and Literature, General Fiction]",1922
4,f5c32f05-2e3c-5eac-baec-7504adf01a18,The Thirteenth Tale,Diane Setterfield,"When her health begins failing, the mysterious...",https://images-na.ssl-images-amazon.com/images...,915,3.99,"[Paper, Audiobook, Ebook]","[Fiction and Literature, General Fiction, Myst...",2006


### Creating `genres_df` & `book_genres_df` for tables

In [5]:
book_genre_long_df = (
    books.loc[:, ['_id', 'genres']]
    .explode('genres')
    .dropna(subset=['genres'])
    .rename(columns={'_id': 'book_id', 'genres': 'genre_name'})
)

book_genre_long_df.head()

,book_id,genre_name
0,73f65a70-8825-5401-bcce-2313da0cc6f8,General Fiction
0,73f65a70-8825-5401-bcce-2313da0cc6f8,Fiction and Literature
0,73f65a70-8825-5401-bcce-2313da0cc6f8,Mystery
1,e940f1b3-d333-50b7-a2b6-b95603134438,General Fiction
1,e940f1b3-d333-50b7-a2b6-b95603134438,Fiction and Literature


In [6]:
unique_genre_names = pd.Index(book_genre_long_df['genre_name'].unique()).sort_values()

genre_name_to_id = {
    genre_name: str(uuid.uuid5(uuid.NAMESPACE_DNS, genre_name))
    for genre_name in unique_genre_names
}

In [7]:
genres_df = (
    pd.DataFrame({
        'id': [genre_name_to_id[genre_name] for genre_name in unique_genre_names],
        'name': unique_genre_names
    })
    .astype({'id': 'string', 'name': 'string'})
)

genres_df.head()

,id,name
0,f4a0080c-b44a-51d3-aebe-42ca4056feca,Anthropology
1,3b4382b7-35e6-5e09-a28f-6475076c62ac,Biography & Memoir
2,b3ab2519-219a-503d-9907-67bb2946d764,Business
3,0be4e68c-f055-54ca-89e4-16462c5999e5,Children's Books
4,237428e6-5467-519f-9570-267b0999f71c,Christian Fiction


In [8]:
book_genres_df = (
    book_genre_long_df
    .assign(genre_id=book_genre_long_df['genre_name'].map(genre_name_to_id))
    .loc[:, ['book_id', 'genre_id']]
    .drop_duplicates()
    .astype({'book_id': 'string', 'genre_id': 'string'})
)

book_genres_df.head()

,book_id,genre_id
0,73f65a70-8825-5401-bcce-2313da0cc6f8,8bd287ce-d503-5dcc-9f04-75f239352d05
0,73f65a70-8825-5401-bcce-2313da0cc6f8,14d90134-849d-538e-b500-d2b2c64d4a6e
0,73f65a70-8825-5401-bcce-2313da0cc6f8,0ff929a9-a1dd-52e4-bcf6-58777381697f
1,e940f1b3-d333-50b7-a2b6-b95603134438,8bd287ce-d503-5dcc-9f04-75f239352d05
1,e940f1b3-d333-50b7-a2b6-b95603134438,14d90134-849d-538e-b500-d2b2c64d4a6e


### `MediaType`

In [9]:
book_media_long_df = (
    books.loc[:, ['_id', 'media_type']]
    .explode('media_type')
    .dropna(subset=['media_type'])
    .rename(columns={'_id': 'book_id', 'media_type': 'media_type_name'})
)

book_media_long_df.head()

,book_id,media_type_name
0,73f65a70-8825-5401-bcce-2313da0cc6f8,Paper
0,73f65a70-8825-5401-bcce-2313da0cc6f8,Audiobook
0,73f65a70-8825-5401-bcce-2313da0cc6f8,Ebook
1,e940f1b3-d333-50b7-a2b6-b95603134438,Paper
1,e940f1b3-d333-50b7-a2b6-b95603134438,Audiobook


In [10]:
unique_media_type_names = pd.Index(book_media_long_df['media_type_name'].unique()).sort_values()

media_type_name_to_id = {
    media_type_name: str(uuid.uuid5(uuid.NAMESPACE_DNS, media_type_name))
    for media_type_name in unique_media_type_names
}

In [11]:
media_types_df = (
    pd.DataFrame({
        'id': [media_type_name_to_id[media_type_name] for media_type_name in unique_media_type_names],
        'name': unique_media_type_names
    })
    .astype({'id': 'string', 'name': 'string'})
)

media_types_df.head()

,id,name
0,0f4a2fdc-ad36-5cab-9498-dbf23bfac0c8,Audiobook
1,db70dabb-2da9-58e4-abd6-906bbc122247,Ebook
2,069af5d9-7041-5eda-9a32-c7c0d61c420f,Paper


In [12]:
book_media_types_df = (
    book_media_long_df
    .assign(media_type_id=book_media_long_df['media_type_name'].map(media_type_name_to_id))
    .loc[:, ['book_id', 'media_type_id']]
    .drop_duplicates()
    .astype({'book_id': 'string', 'media_type_id': 'string'})
)

book_media_types_df.head()

,book_id,media_type_id
0,73f65a70-8825-5401-bcce-2313da0cc6f8,069af5d9-7041-5eda-9a32-c7c0d61c420f
0,73f65a70-8825-5401-bcce-2313da0cc6f8,0f4a2fdc-ad36-5cab-9498-dbf23bfac0c8
0,73f65a70-8825-5401-bcce-2313da0cc6f8,db70dabb-2da9-58e4-abd6-906bbc122247
1,e940f1b3-d333-50b7-a2b6-b95603134438,069af5d9-7041-5eda-9a32-c7c0d61c420f
1,e940f1b3-d333-50b7-a2b6-b95603134438,0f4a2fdc-ad36-5cab-9498-dbf23bfac0c8


### Dropping genres & media_type

In [13]:
books.drop(columns=['genres', 'media_type'], inplace=True)

### Table: books

In [14]:
books.head()

,_id,title,author,description,image_url,reviews_count,rating,year
0,73f65a70-8825-5401-bcce-2313da0cc6f8,Crime and Punishment,Fyodor Dostoevsky,Fyodor Dostoevsky's Crime and Punishment is on...,https://images-na.ssl-images-amazon.com/images...,521,4.25,1866
1,e940f1b3-d333-50b7-a2b6-b95603134438,Love in the Time of Cholera,Gabriel García Márquez,"In their youth, Florentino Ariza and Fermina D...",https://images-na.ssl-images-amazon.com/images...,423,3.95,1985
2,74ec4ed6-0c42-5425-86bf-51af8793a280,Mansfield Park,Jane Austen,Mansfield Park is the study of three families-...,https://images-na.ssl-images-amazon.com/images...,373,3.84,1814
3,8368087f-946e-5eb0-b630-a614124d550f,Siddhartha,Hermann Hesse,Hermann Hesse wrote Siddhartha after he travel...,https://images-na.ssl-images-amazon.com/images...,398,3.95,1922
4,f5c32f05-2e3c-5eac-baec-7504adf01a18,The Thirteenth Tale,Diane Setterfield,"When her health begins failing, the mysterious...",https://images-na.ssl-images-amazon.com/images...,915,3.99,2006


In [15]:
books = books[
    ['_id', 'title', 'author', 'year', 'description', 'image_url', 'reviews_count', 'rating']
]

books.head()

,_id,title,author,year,description,image_url,reviews_count,rating
0,73f65a70-8825-5401-bcce-2313da0cc6f8,Crime and Punishment,Fyodor Dostoevsky,1866,Fyodor Dostoevsky's Crime and Punishment is on...,https://images-na.ssl-images-amazon.com/images...,521,4.25
1,e940f1b3-d333-50b7-a2b6-b95603134438,Love in the Time of Cholera,Gabriel García Márquez,1985,"In their youth, Florentino Ariza and Fermina D...",https://images-na.ssl-images-amazon.com/images...,423,3.95
2,74ec4ed6-0c42-5425-86bf-51af8793a280,Mansfield Park,Jane Austen,1814,Mansfield Park is the study of three families-...,https://images-na.ssl-images-amazon.com/images...,373,3.84
3,8368087f-946e-5eb0-b630-a614124d550f,Siddhartha,Hermann Hesse,1922,Hermann Hesse wrote Siddhartha after he travel...,https://images-na.ssl-images-amazon.com/images...,398,3.95
4,f5c32f05-2e3c-5eac-baec-7504adf01a18,The Thirteenth Tale,Diane Setterfield,2006,"When her health begins failing, the mysterious...",https://images-na.ssl-images-amazon.com/images...,915,3.99


In [16]:
featured_rating_df = pd.DataFrame(
    list(mongo.get_collection('reviews_clean').find({}, {'_id': True, 'featured_rating': True})))
featured_rating_df.head()

,_id,featured_rating
0,73f65a70-8825-5401-bcce-2313da0cc6f8,4.35
1,e940f1b3-d333-50b7-a2b6-b95603134438,3.67
2,74ec4ed6-0c42-5425-86bf-51af8793a280,3.95
3,8368087f-946e-5eb0-b630-a614124d550f,3.85
4,f5c32f05-2e3c-5eac-baec-7504adf01a18,4.26


In [17]:
sentiment_scores_df = pd.DataFrame(list(mongo.get_collection('sentiment_scores').find({})))
sentiment_scores_df['sentiment_score'] = round(sentiment_scores_df['sentiment_score'] * 5, 2)
sentiment_scores_df.head()

,_id,sentiment_score
0,73f65a70-8825-5401-bcce-2313da0cc6f8,4.26
1,e940f1b3-d333-50b7-a2b6-b95603134438,3.63
2,74ec4ed6-0c42-5425-86bf-51af8793a280,3.73
3,8368087f-946e-5eb0-b630-a614124d550f,3.99
4,f5c32f05-2e3c-5eac-baec-7504adf01a18,4.69


In [18]:
summarized_reviews_df = pd.DataFrame(list(mongo.get_collection('summarized_reviews').find({})))
summarized_reviews_df.head()

,_id,summary
0,73f65a70-8825-5401-bcce-2313da0cc6f8,Rodian's motive remains obscured and his effor...
1,e940f1b3-d333-50b7-a2b6-b95603134438,"Venal, cowardly and physically repellent, brag..."
2,74ec4ed6-0c42-5425-86bf-51af8793a280,This is one of my favourite austen novels now....
3,8368087f-946e-5eb0-b630-a614124d550f,"Hermann hesse's book, slaughterhouse five, is ..."
4,f5c32f05-2e3c-5eac-baec-7504adf01a18,"The thirteenth tale is a darkly rich, many lay..."


In [19]:
books_df = books.merge(
    featured_rating_df, how='inner', on='_id'
).merge(
    sentiment_scores_df, how='inner', on='_id'
).merge(
    summarized_reviews_df, how='inner', on='_id'
)

books_df.head()

,_id,title,author,year,description,image_url,reviews_count,rating,featured_rating,sentiment_score,summary
0,73f65a70-8825-5401-bcce-2313da0cc6f8,Crime and Punishment,Fyodor Dostoevsky,1866,Fyodor Dostoevsky's Crime and Punishment is on...,https://images-na.ssl-images-amazon.com/images...,521,4.25,4.35,4.26,Rodian's motive remains obscured and his effor...
1,e940f1b3-d333-50b7-a2b6-b95603134438,Love in the Time of Cholera,Gabriel García Márquez,1985,"In their youth, Florentino Ariza and Fermina D...",https://images-na.ssl-images-amazon.com/images...,423,3.95,3.67,3.63,"Venal, cowardly and physically repellent, brag..."
2,74ec4ed6-0c42-5425-86bf-51af8793a280,Mansfield Park,Jane Austen,1814,Mansfield Park is the study of three families-...,https://images-na.ssl-images-amazon.com/images...,373,3.84,3.95,3.73,This is one of my favourite austen novels now....
3,8368087f-946e-5eb0-b630-a614124d550f,Siddhartha,Hermann Hesse,1922,Hermann Hesse wrote Siddhartha after he travel...,https://images-na.ssl-images-amazon.com/images...,398,3.95,3.85,3.99,"Hermann hesse's book, slaughterhouse five, is ..."
4,f5c32f05-2e3c-5eac-baec-7504adf01a18,The Thirteenth Tale,Diane Setterfield,2006,"When her health begins failing, the mysterious...",https://images-na.ssl-images-amazon.com/images...,915,3.99,4.26,4.69,"The thirteenth tale is a darkly rich, many lay..."


### Users

In [20]:
users_rating_df = pd.DataFrame(list(mongo.get_collection('users_rating').find({})))
users_rating_df.head()

,_id,user_id,username,ratings
0,6898dcc048b31c8bc41a55ed,544c5233-704a-44c3-94ce-865c84d8c09e,Mahdi.Lotfabadi,[{'book_id': '73f65a70-8825-5401-bcce-2313da0c...
1,6898dcc148b31c8bc41a55ee,ccac2163-4237-4e2f-97ca-d0fcc60bf38f,Cecrow,[{'book_id': '73f65a70-8825-5401-bcce-2313da0c...
2,6898dcc148b31c8bc41a55ef,1a8c4728-2dc8-4723-8ed0-812a320132c0,missizicks,[{'book_id': '73f65a70-8825-5401-bcce-2313da0c...
3,6898dcc148b31c8bc41a55f0,4db389b1-150d-4ab5-84aa-75556210f26e,MichaelBarsa,[{'book_id': '73f65a70-8825-5401-bcce-2313da0c...
4,6898dcc148b31c8bc41a55f1,b0d2df12-c24c-4768-8f60-b958731339e0,featherbear,[{'book_id': '73f65a70-8825-5401-bcce-2313da0c...


In [21]:
users_df = (
    users_rating_df[['user_id', 'username']]
    .rename(columns={'user_id': 'id'})
    .drop_duplicates(subset=['id'])
    .astype({'id': 'string', 'username': 'string'})
)

users_df.head()

,id,username
0,544c5233-704a-44c3-94ce-865c84d8c09e,Mahdi.Lotfabadi
1,ccac2163-4237-4e2f-97ca-d0fcc60bf38f,Cecrow
2,1a8c4728-2dc8-4723-8ed0-812a320132c0,missizicks
3,4db389b1-150d-4ab5-84aa-75556210f26e,MichaelBarsa
4,b0d2df12-c24c-4768-8f60-b958731339e0,featherbear


In [22]:
ratings_long_df = (
    users_rating_df[['user_id', 'ratings']]
    .explode('ratings')
    .dropna(subset=['ratings'])
    .reset_index(drop=True)  # <- align index
)

ratings_long_df.head()

,user_id,ratings
0,544c5233-704a-44c3-94ce-865c84d8c09e,{'book_id': '73f65a70-8825-5401-bcce-2313da0cc...
1,544c5233-704a-44c3-94ce-865c84d8c09e,{'book_id': 'e96b3e47-5a70-5a2f-b1cd-bc3ea9fcd...
2,544c5233-704a-44c3-94ce-865c84d8c09e,{'book_id': 'd1b93763-39f1-53ab-bd43-7465188f7...
3,544c5233-704a-44c3-94ce-865c84d8c09e,{'book_id': '9929ad23-b431-5c66-9aa4-79fed6b73...
4,544c5233-704a-44c3-94ce-865c84d8c09e,{'book_id': '426177e2-8662-508b-948e-833ab428b...


In [23]:
ratings_inner = pd.DataFrame(ratings_long_df['ratings'].tolist())

ratings_df = (
    pd.concat([ratings_long_df[['user_id']], ratings_inner], axis=1)
      [['user_id', 'book_id', 'rating']]
      .dropna(subset=['book_id', 'rating'])
)
ratings_df['rating'] = ratings_df['rating'].astype('int16')

ratings_df.head()

,user_id,book_id,rating
0,544c5233-704a-44c3-94ce-865c84d8c09e,73f65a70-8825-5401-bcce-2313da0cc6f8,5
1,544c5233-704a-44c3-94ce-865c84d8c09e,e96b3e47-5a70-5a2f-b1cd-bc3ea9fcd26d,3
2,544c5233-704a-44c3-94ce-865c84d8c09e,d1b93763-39f1-53ab-bd43-7465188f77af,5
3,544c5233-704a-44c3-94ce-865c84d8c09e,9929ad23-b431-5c66-9aa4-79fed6b73eb0,3
4,544c5233-704a-44c3-94ce-865c84d8c09e,426177e2-8662-508b-948e-833ab428be75,5


### Upload to postgres db

In [24]:
def load_table(df, cols, sql):
    """Exec a parameterized INSERT for df[cols] using executemany."""
    if df.empty:
        print("skip (empty):", sql.split()[2])
        return
    rows = list(df[cols].itertuples(index=False, name=None))
    postgres.executemany(sql, rows)
    print(f"inserted {len(rows)} rows into {sql.split()[2]}")

In [25]:
# ---------- 1) users ----------
# users_df columns: ['id','username']
load_table(
    users_df,
    ['id','username'],
    """
    INSERT INTO library.users (id, username)
    VALUES (%s, %s)
    ON CONFLICT (id) DO NOTHING
    """
)

inserted 7381 rows into library.users


In [26]:
# ---------- 2) books ----------
# books_df columns (in this order):
# ['_id','title','author','year','description','image_url','reviews_count', 'rating','featured_rating','sentiment_score','summary']
load_table(
    books_df,
    ['_id','title','author','year','description','image_url',
     'reviews_count','rating','featured_rating','sentiment_score','summary'],
    """
    INSERT INTO library.books (
        id, title, author, "year", description, image_url, reviews_count,
        rating, featured_rating, sentiment_score, summary
    )
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
    ON CONFLICT (id) DO NOTHING
    """
)

inserted 228 rows into library.books


In [27]:
# ---------- 3) genres ----------
# genres_df columns: ['id','name']
load_table(
    genres_df,
    ['id','name'],
    """
    INSERT INTO library.genres (id, name)
    VALUES (%s, %s)
    ON CONFLICT (id) DO NOTHING
    """
)

inserted 31 rows into library.genres


In [28]:
# ---------- 4) media_types ----------
# media_types_df columns: ['id','name']
load_table(
    media_types_df,
    ['id','name'],
    """
    INSERT INTO library.media_types (id, name)
    VALUES (%s, %s)
    ON CONFLICT (id) DO NOTHING
    """
)

inserted 3 rows into library.media_types


In [29]:
# ---------- 5) book_genres (M:N) ----------
# book_genres_df columns: ['book_id','genre_id']
load_table(
    book_genres_df,
    ['book_id','genre_id'],
    """
    INSERT INTO library.book_genres (book_id, genre_id)
    VALUES (%s, %s)
    ON CONFLICT (book_id, genre_id) DO NOTHING
    """
)

inserted 704 rows into library.book_genres


In [30]:
# ---------- 6) book_media_types (M:N) ----------
# book_media_types_df columns: ['book_id','media_type_id']
load_table(
    book_media_types_df,
    ['book_id','media_type_id'],
    """
    INSERT INTO library.book_media_types (book_id, media_type_id)
    VALUES (%s, %s)
    ON CONFLICT (book_id, media_type_id) DO NOTHING
    """
)

inserted 684 rows into library.book_media_types


In [31]:
# ---------- 7) ratings ----------
# ratings_df columns: ['user_id','book_id','rating']
# (ensure rating is int 1..5 before load)
ratings_df['rating'] = ratings_df['rating'].astype('int16')
load_table(
    ratings_df,
    ['user_id','book_id','rating'],
    """
    INSERT INTO library.rating (user_id, book_id, rating)
    VALUES (%s, %s, %s)
    ON CONFLICT (user_id, book_id) DO NOTHING
    """
)

inserted 21634 rows into library.rating


---